In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from pandas import DataFrame
from pandas import Series

In [30]:
# read in data
df_title_basics = pd.read_csv('title.basics.tsv', sep='\t', na_values=['\\N'])
df_principals = pd.read_csv('title.principals.tsv', sep='\t')
df_title_ratings = pd.read_csv('title.ratings.tsv', sep='\t')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_title_basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0.0,1894.0,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0.0,1892.0,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0.0,1892.0,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0.0,1892.0,NaN,NaN,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0.0,1893.0,NaN,1,Short


In [4]:
df_title_ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.8,1310
1,tt0000002,6.5,149
2,tt0000003,6.6,888
3,tt0000004,6.4,92
4,tt0000005,6.2,1569


In [31]:
# combine title_ratings and title_basics
df_title_basics_ratings = pd.merge(left=df_title_basics,right=df_title_ratings, how='left',\
                       left_on='tconst', right_on='tconst')

df_title_basics_ratings=df_title_basics_ratings.loc[:,['tconst','titleType','primaryTitle','runtimeMinutes','genres', 'averageRating', 'numVotes']]

df_title_basics_ratings=df_title_basics_ratings[df_title_basics_ratings.titleType=='movie']
df_title_basics_ratings=df_title_basics_ratings.dropna(subset=['numVotes','runtimeMinutes'])

df_title_basics_ratings.head()

,tconst,titleType,primaryTitle,runtimeMinutes,genres,averageRating,numVotes
8,tt0000009,movie,Miss Jerry,45,Romance,5.4,58.0
145,tt0000147,movie,The Corbett-Fitzsimmons Fight,20,"Documentary,News,Sport",5.2,236.0
571,tt0000574,movie,The Story of the Kelly Gang,70,"Biography,Crime,Drama",6.4,413.0
581,tt0000584,movie,El ciego de la aldea,75,Drama,4.8,35.0
673,tt0000679,movie,The Fairylogue and Radio-Plays,120,"Adventure,Fantasy",6.1,24.0


In [32]:
# export merged titles and ratings as csv

df_title_basics_ratings.to_csv('title.ratings.genreG.actorN.csv', sep = '\t', encoding = 'utf-8')

In [33]:
# separate genres (1 per movie)

df_genre= df_title_basics_ratings.set_index('tconst')\
    .genres.str.split(',', expand = True)\
    .stack().reset_index('tconst')\
    .rename(columns={0:'genre'})
    
# merge back
df_title_basics_ratings2 = pd.merge(left=df_genre,\
                                   right=df_title_basics_ratings, \
                                   how='left',\
                                   left_on='tconst',\
                                   right_on='tconst')

df_title_basics_ratings2 = df_title_basics_ratings2.drop('genres', axis=1)
df_title_basics_ratings2.head()

,tconst,genre,titleType,primaryTitle,runtimeMinutes,averageRating,numVotes
0,tt0000009,Romance,movie,Miss Jerry,45,5.4,58.0
1,tt0000147,Documentary,movie,The Corbett-Fitzsimmons Fight,20,5.2,236.0
2,tt0000147,News,movie,The Corbett-Fitzsimmons Fight,20,5.2,236.0
3,tt0000147,Sport,movie,The Corbett-Fitzsimmons Fight,20,5.2,236.0
4,tt0000574,Biography,movie,The Story of the Kelly Gang,70,6.4,413.0


In [34]:
# export merged titles and ratings and separated genres as csv

df_title_basics_ratings2.to_csv('title.ratings.genreS.actorN.csv', sep = '\t', encoding = 'utf-8')

In [9]:
df_principals.head()

,tconst,principalCast
0,tt0000001,"nm0374658,nm0005690,nm1588970"
1,tt0000002,"nm0721526,nm1335271"
2,tt0000003,"nm5442200,nm1335271,nm0721526,nm5442194"
3,tt0000004,"nm1335271,nm0721526"
4,tt0000005,"nm0005690,nm0443482,nm0653042"


In [35]:
# separate principals one actor per row
df_principals2= df_principals.set_index('tconst')\
    .principalCast.str.split(',', expand = True)\
    .stack().reset_index('tconst')\
    .rename(columns={0:'principalCast'})

In [11]:
df_principals2.head()

,tconst,principalCast
0,tt0000001,nm0374658
1,tt0000001,nm0005690
2,tt0000001,nm1588970
0,tt0000002,nm0721526
1,tt0000002,nm1335271


In [36]:
# merge dfs and export to csv

# merge actors (genres grouped, actors grouped)
df = pd.merge(df_principals, df_title_basics_ratings,\
              how='right', on='tconst')
df.to_csv('title.ratings.genresG.actorsG.csv', sep = '\t', encoding = 'utf-8')

# merge actors (genres separated, actors grouped)
df2 = pd.merge(df_principals, df_title_basics_ratings2,\
               how='right', on='tconst')
df2.to_csv('title.ratings.genresS.actorsG.csv', sep = '\t', encoding = 'utf-8')

# merge actors (genres grouped, actors separated)
df3 = pd.merge(df_principals2, df_title_basics_ratings,\
               how='right', on='tconst')
df3.to_csv('title.ratings.genresG.actorsS.csv', sep = '\t', encoding = 'utf-8')

# merge actors (genres separated, actors separated)
# df4 = pd.merge(df_principals2, df_title_basics_ratings2,\
#               how='right', on='tconst')
# d4.to_csv('merged_genresS_actorsS.csv', sep = '\t', encoding = 'utf-8')

In [13]:
df.head()

,tconst,principalCast,titleType,primaryTitle,runtimeMinutes,genres,averageRating,numVotes
0,tt0000009,"nm0085156,nm0183823,nm0063086,nm1309758",movie,Miss Jerry,45,Romance,5.4,58.0
1,tt0000147,"nm0280615,nm4082222,nm2256592,nm0103755,nm0714...",movie,The Corbett-Fitzsimmons Fight,20,"Documentary,News,Sport",5.2,236.0
2,tt0000574,"nm2421834,nm0170118,nm0846911,nm0846894,nm0425...",movie,The Story of the Kelly Gang,70,"Biography,Crime,Drama",6.4,413.0
3,tt0000584,"nm0191133,nm0305591,nm0400103",movie,El ciego de la aldea,75,Drama,4.8,35.0
4,tt0000679,"nm0000875,nm0933446,nm0122665,nm0877783,nm0542...",movie,The Fairylogue and Radio-Plays,120,"Adventure,Fantasy",6.1,24.0


In [14]:
df2.head()

,tconst,principalCast,genre,titleType,primaryTitle,runtimeMinutes,averageRating,numVotes
0,tt0000009,"nm0085156,nm0183823,nm0063086,nm1309758",Romance,movie,Miss Jerry,45,5.4,58.0
1,tt0000147,"nm0280615,nm4082222,nm2256592,nm0103755,nm0714...",Documentary,movie,The Corbett-Fitzsimmons Fight,20,5.2,236.0
2,tt0000147,"nm0280615,nm4082222,nm2256592,nm0103755,nm0714...",News,movie,The Corbett-Fitzsimmons Fight,20,5.2,236.0
3,tt0000147,"nm0280615,nm4082222,nm2256592,nm0103755,nm0714...",Sport,movie,The Corbett-Fitzsimmons Fight,20,5.2,236.0
4,tt0000574,"nm2421834,nm0170118,nm0846911,nm0846894,nm0425...",Biography,movie,The Story of the Kelly Gang,70,6.4,413.0


In [15]:
df3.head()

,tconst,principalCast,titleType,primaryTitle,runtimeMinutes,genres,averageRating,numVotes
0,tt0000009,nm0085156,movie,Miss Jerry,45,Romance,5.4,58.0
1,tt0000009,nm0183823,movie,Miss Jerry,45,Romance,5.4,58.0
2,tt0000009,nm0063086,movie,Miss Jerry,45,Romance,5.4,58.0
3,tt0000009,nm1309758,movie,Miss Jerry,45,Romance,5.4,58.0
4,tt0000147,nm0280615,movie,The Corbett-Fitzsimmons Fight,20,"Documentary,News,Sport",5.2,236.0
